In [4]:
import os
import torch
import json
import numpy as np
from PIL import Image
from scipy.ndimage import label as cc_label
import time
from google.colab import drive
from tqdm import tqdm # For progress bar tracking


drive.mount('/content/drive', force_remount=True)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

BDD_ROOT = "/content/drive/MyDrive/Latitude_AI_Team/data"
MASK_DIR = f"{BDD_ROOT}/bdd100k_segmentations/bdd100k_seg_maps_2/labels/train"
PREPROCESSED_JSON = f"{BDD_ROOT}/bdd100k_preprocessed_masks.json"

BDD_IDS = [6, 7, 11, 13, 12, 14, 15, 17, 18] # Your Thing IDs
ID_TO_LABEL = {bdd_id: i+1 for i, bdd_id in enumerate(BDD_IDS)}
MIN_AREA = 200
MAX_AREA_RATIO = 0.5


def run_preprocessor(mask_dir, output_json):
    """Calculates all complex instance data once and saves it to a JSON file."""

    masks_files = sorted([f for f in os.listdir(mask_dir) if f.lower().endswith(('.png', '.jpg'))])

    data_map = {}

    print(f"Starting complex data pre-processing for {len(masks_files)} mask files...")

    # Use tqdm for a clear progress bar that shows the utility working
    for mask_file in tqdm(masks_files):
        mask_path = os.path.join(mask_dir, mask_file)

        try:
            mask_pil = Image.open(mask_path).convert('L')
            mask_np = np.array(mask_pil)

            H, W = mask_np.shape
            max_allowed_area = H * W * MAX_AREA_RATIO

            # List to store results for this single image
            image_instances = []

            for bdd_id in BDD_IDS:
                class_mask = (mask_np == bdd_id).astype(np.uint8)

                if class_mask.sum() == 0: continue

                inst_map, ninst = cc_label(class_mask)

                for inst_id in range(1, ninst + 1):
                    inst_mask = (inst_map == inst_id)
                    current_area = inst_mask.sum()

                    if current_area < MIN_AREA: continue
                    if current_area > max_allowed_area: continue

                    ys, xs = np.where(inst_mask)
                    xmin, ymin, xmax, ymax = xs.min(), ys.min(), xs.max(), ys.max()

                    # Store data in a simple, savable format (list of integers for box)
                    image_instances.append({
                        "box": [int(xmin), int(ymin), int(xmax), int(ymax)],
                        "label": ID_TO_LABEL[bdd_id],
                        # NOTE: We skip saving the full mask pixels to save JSON size
                        # We rely on the bounding box and class for initial training success
                        "area": int(current_area)
                    })

            # Map the original image name to the instance list
            base_name = os.path.splitext(mask_file)[0].replace("_train_id", "")
            data_map[base_name] = image_instances

        except Exception as e:
            print(f"Error processing {mask_file}: {e}")
            continue

    # Save the resulting map to a JSON file
    with open(output_json, 'w') as f:
        json.dump(data_map, f)

    print(f"Pre-processing complete. Saved final JSON to: {output_json}")


print("Running pre-processor...")
run_preprocessor(MASK_DIR, PREPROCESSED_JSON)
print("Pre-processor finished. Now launch training script.")

Mounted at /content/drive
Running pre-processor...
Starting complex data pre-processing for 7000 mask files...


100%|██████████| 7000/7000 [15:18<00:00,  7.63it/s]


Pre-processing complete. Saved final JSON to: /content/drive/MyDrive/Latitude_AI_Team/data/bdd100k_preprocessed_masks.json
Pre-processor finished. Now launch training script.
